In this notebook we'll explore topic modeling to discover broad themes in a collection of movie summaries.

In [1]:
import nltk
import re
import gensim
from gensim import corpora
import operator

nltk.download('stopwords')
from nltk.corpus import stopwords

import numpy as np
import random

random.seed(1)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/filipesantos/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def read_stopwords(filename):
    stopwords={}
    with open(filename) as file:
        for line in file:
            stopwords[line.rstrip()]=1
    return stopwords

Since we're running topic modeling on texts with lots of names, we'll add the Jockers list of stopwords (which includes character names) to our stoplist.

In [3]:
stop_words = {k:1 for k in stopwords.words('english')}
stop_words.update(read_stopwords("../data/jockers.stopwords"))
stop_words["'s"]=1
stop_words=list(stop_words.keys())

In [4]:
def filter(word, stopwords):
    
    """ Function to exclude words from a text """
    
    # no stopwords
    if word in stopwords:
        return False
    
    # has to contain at least one letter
    if re.search("[A-Za-z]", word) is not None:
        return True
    
    return False

In [5]:
def read_docs(plotFile, metadataFile, stopwords):
    
    names={}
    box={}
    
    with open(metadataFile, encoding="utf-8") as file:
        for line in file:
            cols=line.rstrip().split("\t")
            idd=cols[0]
            name=cols[2]
            boxoffice=cols[4]
            if len(boxoffice) != 0:
                box[idd]=int(boxoffice)
                names[idd]=name
    
    n=5000
    target_movies={}


    sorted_box = sorted(box.items(), key=operator.itemgetter(1), reverse=True)
    for k, v in sorted_box[:n]:
        target_movies[k]=names[k]
    
    docs=[]
    names=[]
   
    with open(plotFile, encoding="utf-8") as file:
        for line in file:
            cols=line.rstrip().split("\t")
            idd=cols[0]
            text=cols[1]
            
            if idd in target_movies:
                tokens=nltk.word_tokenize(text.lower())
                tokens=[x for x in tokens if filter(x, stopwords)]
                docs.append(tokens)
                name=target_movies[idd]
                names.append(name)
    return docs, names

We'll read in summaries of the 5,000 movies with the highest box office revenues.

In [6]:
metadataFile="../data/movie.metadata.tsv"
plotFile="../data/plot_summaries.txt"
data, doc_names=read_docs(plotFile, metadataFile, stop_words)

We will convert the movie summaries into a bag-of-words representation using gensim's [corpora.dictionary](https://radimrehurek.com/gensim/corpora/dictionary.html) methods.

In [14]:
# Create vocab from data; restrict vocab to only the top 10K terms that show up in at least 5 documents 
# and no more than 50% of all documents

dictionary = corpora.Dictionary(data)
dictionary.filter_extremes(no_below=5, no_above=.5, keep_n=10000)

Dictionary<10000 unique tokens: ['12-year-old', 'able', 'accompanied', 'advice', 'afterward']...>


In [15]:
# Replace dataset with numeric ids words in vocab (and exclude all other words)
corpus = [dictionary.doc2bow(text) for text in data]

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [9]:
num_topics=20

Now let's run a topic model on this data using gensim's built-in LDA.

In [10]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=num_topics, 
                                           passes=10,
                                           alpha='auto')

We can get a sense of what the topics are by printing the top 10 words with highest $P(word \mid topic)$ for each topic

In [11]:
for i in range(num_topics):
    print("topic %s:\t%s" % (i, ' '.join([term for term, freq in lda_model.show_topic(i, topn=10)])))

topic 0:	japanese boat camp water tank shark connor english holmes mask
topic 1:	house finds find room tells goes body tries night car
topic 2:	men town war president american west government army general wife
topic 3:	bond turner flynn agent frost griffin campbell skull sir nixon
topic 4:	case judge court shaw trial law firm attorney sin company
topic 5:	life film mother family time father begins story years finds
topic 6:	earth ship alien planet space robot crew aliens nuclear miller
topic 7:	find city battle castle escape group world fight kill return
topic 8:	dracula collins toys hayes machine cutter race apprentice count armstrong
topic 9:	team game play duke coach football win player players playing
topic 10:	tells show relationship n't father wedding asks job family band
topic 11:	captain plane mission men killed team crew bomb orders officer
topic 12:	banks ally powell hawk gin harvard paulie triangle clock bank
topic 13:	dr. kill creature killed vampire infected blood killing 

Another way of understanding topics is to print out the documents that have the highest topic representation -- i.e., for a given topic $k$, the documents with highest $P(topic=k | document)$.  How much do the documents listed here align with your understanding of the topics?

In [12]:
topic_model=lda_model 

topic_docs=[]
for i in range(num_topics):
    topic_docs.append({})
for doc_id in range(len(corpus)):
    doc_topics=topic_model.get_document_topics(corpus[doc_id])
    for topic_num, topic_prob in doc_topics:
        topic_docs[topic_num][doc_id]=topic_prob

for i in range(num_topics):
    print("%s\n" % ' '.join([term for term, freq in topic_model.show_topic(i, topn=10)]))
    sorted_x = sorted(topic_docs[i].items(), key=operator.itemgetter(1), reverse=True)
    for k, v in sorted_x[:5]:
        print("%s\t%.3f\t%s" % (i,v,doc_names[k]))
    print()
    
    

japanese boat camp water tank shark connor english holmes mask

0	0.660	Jaws 2
0	0.655	Elizabeth: The Golden Age
0	0.639	Shark Night 3D
0	0.632	Without a Clue
0	0.568	Jaws

house finds find room tells goes body tries night car

1	0.985	Paranormal Activity 4
1	0.977	Monster House
1	0.927	The Collector
1	0.912	Friday the 13th Part 3: 3D
1	0.898	Paranormal Activity 2

men town war president american west government army general wife

2	0.969	Il Divo
2	0.893	The Distinguished Gentleman
2	0.878	Appaloosa
2	0.874	Fair Game
2	0.844	First Family

bond turner flynn agent frost griffin campbell skull sir nixon

3	0.453	Casino Royale
3	0.390	From Russia with Love
3	0.388	Never Say Never Again
3	0.337	GoldenEye
3	0.291	Diamonds Are Forever

case judge court shaw trial law firm attorney sin company

4	0.547	The Devil's Advocate
4	0.508	Rollover
4	0.406	The Verdict
4	0.375	Guilty as Sin
4	0.374	The Associate

life film mother family time father begins story years finds

5	0.981	Once Around
5	0.968	B

In [16]:
print(lda_model.print_topics())

[(0, '0.015*"japanese" + 0.014*"boat" + 0.014*"camp" + 0.011*"water" + 0.009*"tank" + 0.008*"shark" + 0.007*"connor" + 0.007*"english" + 0.006*"holmes" + 0.006*"mask"'), (1, '0.011*"house" + 0.007*"finds" + 0.006*"find" + 0.006*"room" + 0.006*"tells" + 0.005*"goes" + 0.005*"body" + 0.005*"tries" + 0.004*"night" + 0.004*"car"'), (2, '0.008*"men" + 0.006*"town" + 0.006*"war" + 0.006*"president" + 0.004*"american" + 0.004*"west" + 0.004*"government" + 0.004*"army" + 0.004*"general" + 0.003*"wife"'), (3, '0.137*"bond" + 0.042*"turner" + 0.022*"flynn" + 0.019*"agent" + 0.018*"frost" + 0.018*"griffin" + 0.017*"campbell" + 0.016*"skull" + 0.015*"sir" + 0.014*"nixon"'), (4, '0.032*"case" + 0.028*"judge" + 0.024*"court" + 0.024*"shaw" + 0.019*"trial" + 0.017*"law" + 0.016*"firm" + 0.015*"attorney" + 0.014*"sin" + 0.013*"company"'), (5, '0.011*"life" + 0.009*"film" + 0.009*"mother" + 0.008*"family" + 0.007*"time" + 0.007*"father" + 0.005*"begins" + 0.005*"story" + 0.005*"years" + 0.004*"finds"')

In [20]:
import pyLDAvis
import pyLDAvis.gensim  # don't skip this

In [19]:
pip install pyLDAvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 17.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for fst-pso: filename=fst_pso-1.8.1-py3-none-any.whl size=20430 sha256=f16d5940aafb9ee8d1890fcffe1c5fe41f22e952e14ac1b350a62713bdac34f7
  Stored in directory: /Users/filipesantos/Library/Caches/pip/wheels/69/f5/e5/18ad53fe1ed6b2af9fad05ec052e4acbac8e92441df44bad2e
  Created wheel for miniful: filename=miniful-0.0.6-py3-none-any.whl size=3513 sha256=665e8a678e90aedd21a6c4c93efa2d8eb8c39f34810d2ae75e3853e613fd8c20
  Stored in directory: /Users/filipesantos/Library/Caches/pip/wheels/9d/ff/2f/afe4cd56f47de147407705626517d68bea0f3b74eb1fb168e6
Successfully built fst-pso miniful
Note: you may need to restart the kernel to use updated packages.


In [22]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.076211  0.021439       1        1  14.215675
10    -0.125393 -0.134880       2        1  11.842290
18    -0.122597 -0.088027       3        1  11.296554
5     -0.093739 -0.091759       4        1  10.850556
7     -0.055081  0.093564       5        1   9.023979
15    -0.108185 -0.021270       6        1   7.198725
11    -0.023994  0.166409       7        1   6.841785
2     -0.065215  0.006019       8        1   6.478678
17    -0.083373  0.089175       9        1   4.921985
16    -0.092297 -0.098384      10        1   4.486113
6      0.041018  0.206789      11        1   3.359026
13    -0.013449  0.184409      12        1   2.377454
9     -0.081277 -0.104797      13        1   2.149374
0      0.007204  0.098711      14        1   1.572455
14    -0.043481 -0.032605      15        1   1.308428
4      0.075263 -0.141695      16        1   0.854981
3      0.232028  0.062837      17        1   0.440615
19     0.146338 -0.001180      18        1   0.410855
8      0.296798 -0.086402      19        1   0.304819
12     0.185643 -0.128352      20        1   0.065652, topic_info=           Term         Freq        Total Category  logprob  loglift
496      school  2116.000000  2116.000000  Default  30.0000  30.0000
325      police  2875.000000  2875.000000  Default  29.0000  29.0000
2058       team  1791.000000  1791.000000  Default  28.0000  28.0000
870        game  1096.000000  1096.000000  Default  27.0000  27.0000
2801       town  1577.000000  1577.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
4843       bank     6.747330   497.590434  Topic20  -4.6223   3.0279
6123  alcoholic     2.059707   123.238177  Topic20  -5.8088   3.2370
1883  terrorist     1.845600   127.635190  Topic20  -5.9186   3.0922
655         kid     2.236367   222.022901  Topic20  -5.7266   2.7306
415      father     2.991901  4083.503741  Topic20  -5.4355   0.1098

[1347 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
6409      1  0.020255  1950s
6409      4  0.850706  1950s
6409      8  0.101275  1950s
6600      3  0.034937    1st
6600      7  0.174685    1st
...     ...       ...    ...
5446     10  0.100267   zero
5446     11  0.211676   zero
5446     12  0.601604   zero
5446     14  0.066845   zero
8653     18  0.986252   zeus

[5649 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 11, 19, 6, 8, 16, 12, 3, 18, 17, 7, 14, 10, 1, 15, 5, 4, 20, 9, 13])